Design of a new game, Spuds

In [1]:
using Random
using DataFrames
using CSV
using Statistics
using Printf
using LinearAlgebra

In [2]:
struct Spud
    name::String
    h::Int64
    f::Int64
    l::Int64
    p::Int64
    r::Int64
    s::Int64
end

const MXS = 100
const MXV = 20
const MNV = 1

function cost(h::Int64, f::Int64, l::Int64, p::Int64, r::Int64, s::Int64)::Int64
    #return h*(f+l-2*MNV) + f*(p+r+s-3*MNV) + h+f+l+p+r+s-6*MNV
    return h*(f+l) + f*(p+r+s)
end

cost (generic function with 1 method)

In [3]:
function eval_battle(a::Spud, b::Spud)::Int
    a_finds = a.f >= b.h
    b_finds = b.f >= a.h
    #score_a = 4 * (1000*a.p + a.l > 1000*b.p+b.l) + 3 * (1000*a.r+a.l > 1000*b.r+b.l) + 2 * (1000*a.s+a.l > 1000*b.s+b.l)
    #score_b = 4 * (1000*a.p + a.l < 1000*b.p+b.l) + 3 * (1000*a.r+a.l < 1000*b.r+b.l) + 2 * (1000*a.s+a.l < 1000*b.s+b.l)
    score_a = 4 * (a.p > b.p) + 3 * (a.r > b.r) + 2 * (a.s > b.s)
    score_b = 4 * (a.p < b.p) + 3 * (a.r < b.r) + 2 * (a.s < b.s)
    melee_win = sign(score_a - score_b)
    if a_finds && b_finds
        tiebreaker = 4 * melee_win + 2 * sign(a.l - b.l) + sign(a.h - b.h)
        return sign(tiebreaker)
    end
    if a_finds && !b_finds
        return 1
    end
    if !a_finds && b_finds
        return -1
    end
    if !a_finds && !b_finds
        tiebreaker = 4 * sign(a.l - b.l) + 2 * melee_win + sign(a.f - b.f)
        return sign(tiebreaker)
    end
end


eval_battle (generic function with 1 method)

In [4]:
tab = CSV.read("census_yob2022_names.txt", DataFrame, header = false)
names = tab.Column1
adjectives = CSV.read("adjectives.csv", DataFrame)
nouns = CSV.read("nouns.csv", DataFrame)
jobs = CSV.read("jobs.csv", DataFrame)

const mult_noun = 1
const mult_adj = 2
const mult_job = 3


function random_name_and_stat()::Spud
    vp = [0, 0, 0, 0, 0, 0]
    #nametype = rand([1,1,1,1,1,2,2,2,3])
    name = ""
    noun = ""
    adj = ""
    noun_i = rand(1:nrow(nouns))
    noun = nouns[noun_i, :noun]
    vp[1] = vp[1] + nouns[noun_i, :H] * mult_noun
    vp[2] = vp[2] + nouns[noun_i, :F] * mult_noun
    vp[3] = vp[3] + nouns[noun_i, :L] * mult_noun
    vp[4] = vp[4] + nouns[noun_i, :P] * mult_noun
    vp[5] = vp[5] + nouns[noun_i, :R] * mult_noun
    vp[6] = vp[6] + nouns[noun_i, :S] * mult_noun
    adj_i = rand(1:nrow(adjectives))
    adj = adjectives[adj_i, :adjective]
    vp[1] = vp[1] + adjectives[adj_i, :H] * mult_adj
    vp[2] = vp[2] + adjectives[adj_i, :F] * mult_adj
    vp[3] = vp[3] + adjectives[adj_i, :L] * mult_adj
    vp[4] = vp[4] + adjectives[adj_i, :P] * mult_adj
    vp[5] = vp[5] + adjectives[adj_i, :R] * mult_adj
    vp[6] = vp[6] + adjectives[adj_i, :S] * mult_adj
    job_i = rand(1:nrow(jobs))
    job = jobs[job_i, :job]
    vp[1] = vp[1] + jobs[job_i, :H] * mult_job
    vp[2] = vp[2] + jobs[job_i, :F] * mult_job
    vp[3] = vp[3] + jobs[job_i, :L] * mult_job
    vp[4] = vp[4] + jobs[job_i, :P] * mult_job
    vp[5] = vp[5] + jobs[job_i, :R] * mult_job
    vp[6] = vp[6] + jobs[job_i, :S] * mult_job
    name = string(adj, " ", job, " ", noun)
    Spud(name, vp[1], vp[2], vp[3], vp[4], vp[5], vp[6])
end

random_name_and_stat (generic function with 1 method)

In [5]:
function rand_rename(a::Spud, n_tries::Int = 100)::Spud
    best_score = 0.0
    best_b = random_name_and_stat()
    for ii in 1:n_tries
        b = random_name_and_stat()
        b_norm = sqrt(b.h^2 + b.f^2 + b.l^2 + b.p^2 + b.r^2 + b.s^2)
        score = (a.h * b.h + a.f * b.f + a.l * b.l + a.p * b.p + a.r * b.r + a.s * b.s)/b_norm
        if score > best_score
            best_score = score
            best_b = b
        end
    end
    return Spud(best_b.name, a.h, a.f, a.l, a.p, a.r, a.s)
end

rand_rename (generic function with 2 methods)

In [6]:
function eval_battle_list(a::Spud, bs::Array{Spud})::Int
    score = 0
    for ii in 1:length(bs)
        score = score + eval_battle(a, bs[ii])
    end
    return score
end

function eval_battle_list2(a::Spud, bs::Array{Spud}, w::Vector{Float64})::AbstractFloat
    score = 0.0
    for ii in 1:length(bs)
        score = score + w[ii] * eval_battle(a, bs[ii])
    end
    return score
end


function pick_best(as::Array{Spud}, bs::Array{Spud})::Spud
    bestscore = -999
    bestf = as[1]
    for ii in 1:length(as)
        score = eval_battle_list(as[ii], bs)
        if score > bestscore
            bestscore = score
            bestf = as[ii]
        end
    end
    return bestf
end

function pick_best_rdmly(as::Array{Spud}, bs::Array{Spud}, ntries::Int)::Spud
    bestscore = -999
    bestf = rand(as)
    for ii in 1:ntries
        f = rand(as)
        score = eval_battle_list(f, bs)
        if score > bestscore
            bestscore = score
            bestf = f
        end
    end
    return bestf
end

function pick_best_rdmly_g(g::Function, bs::Array{Spud}, ntries::Int)::Spud
    bestscore = -999
    bestf = g()
    for ii in 1:ntries
        f = g()
        score = eval_battle_list(f, bs)
        if score > bestscore
            bestscore = score
            bestf = f
        end
    end
    return bestf
end

function spuds_to_df(as::Array{Spud})::DataFrame
    names = Array{String}(undef, length(as))
    hs = Array{Int}(undef, length(as))
    fs = Array{Int}(undef, length(as))
    ls = Array{Int}(undef, length(as))
    ps = Array{Int}(undef, length(as))
    rs = Array{Int}(undef, length(as))
    ss = Array{Int}(undef, length(as))
    for ii in 1:length(as)
        names[ii] = as[ii].name
        hs[ii] = as[ii].h
        fs[ii] = as[ii].f
        ls[ii] = as[ii].l
        ps[ii] = as[ii].p
        rs[ii] = as[ii].r
        ss[ii] = as[ii].s
    end
    df = DataFrame(name = names, h = hs, f = fs, l = ls, p = ps, r = rs, s = ss)
    return df
end

function fpart(x::AbstractFloat)::AbstractFloat
  return x - trunc(x)
end

function eval_team_battle(as::Array{Spud}, bs::Array{Spud})::Int
    a_i = 1
    b_i = 1
    while (a_i <= length(as)) && (b_i <= length(bs))
        res = eval_battle(as[a_i], bs[b_i])
        if res == 1
            b_i = b_i + 1
        else
            a_i = a_i + 1
            if res == 0
                b_i = b_i + 1
            end
        end
    end
    a_out = (a_i > length(as))
    b_out = (b_i > length(as))
    if a_out
        if b_out
            return 0
        else
            return -1
        end
    else
        return 1
    end
end

function compare_generator(f1, f2, limit)
    a_i = 1
    b_i = 1
    f_a = f1()
    f_b = f2()
    while (a_i < limit) && (b_i < limit)
        res = eval_battle(f_a, f_b)
        if res != -1
            b_i = b_i + 1
            f_b = f2()
        end
        if res != 1
            a_i = a_i + 1
            f_a = f1()
        end
    end
    return (a_i/limit, b_i/limit)
end

function random_team(f::Function, team_size::Int)::Array{Spud}
    team = Array{Spud}(undef, team_size)
    for i in 1:team_size
        team[i] = f()
    end
    return team
end

random_team (generic function with 1 method)

In [7]:
function upgrade_spud(sp::Spud)::Spud
    h = sp.h
    f = sp.f
    l = sp.l
    p = sp.p
    r = sp.r
    s = sp.s    
    check_h = (f == MXV) || (cost(h,f+1,l,p,r,s) > MXS)
    check_f = (h == MXV) || (cost(h+1,f,l,p,r,s) > MXS)
    check_l = (l == MXV) || (cost(h,f,l+1,p,r,s) > MXS)
    check_prs = (p+r+s == 3*MXV) || (cost(h,f,l,p+1,r,s) > MXS)
    while !(check_h && check_f && check_l && check_prs)
        rand_i = rand(1:4)
        if rand_i == 1 && !check_h
            h = h+1
        end
        if rand_i == 2 && !check_f
            f = f+1
        end
        if rand_i == 3 && !check_l
            l = l+1
        end
        if rand_i == 4 && !check_prs
            rand_j = rand([1,1,1,1,1,2,2,2,2,3,3,3])
            if rand_j == 1 && p <= MXV
                p += 1
            end
            if rand_j == 2 && r <= MXV
                r += 1
            end
            if rand_j == 3 && s <= MXV
                s += 1
            end
        end            
        check_h = (f == MXV) || (cost(h,f+1,l,p,r,s) > MXS)
        check_f = (h == MXV) || (cost(h+1,f,l,p,r,s) > MXS)
        check_l = (l == MXV) || (cost(h,f,l+1,p,r,s) > MXS)
        check_prs = (p+r+s == 3*MXV) || (cost(h,f,l,p+1,r,s) > MXS)
    end
    return Spud(sp.name, h, f, l, p, r, s)
end



function pick_best_library(bs::Array{Spud})::Spud
    bestscore = -999
    bestf = Spud("",MNV,MNV,MNV,MNV,MNV,MNV)
    df = spuds_to_df(bs)
    hrange = vcat([MNV], df.h, df.h .+ 1, df.f, df.f .+ 1)
    frange = vcat([MNV], df.f, df.f .+ 1, df.h, df.h .+ 1)
    lrange = vcat([MNV], df.l, df.l .+ 1)
    prange = vcat([MNV], df.p, df.p .+ 1)
    rrange = vcat([MNV], df.r, df.r .+ 1)
    srange = vcat([MNV], df.s, df.s .+ 1)
    hrange = sort(unique(hrange))
    frange = sort(unique(frange))
    lrange = sort(unique(lrange))
    prange = sort(unique(prange))
    rrange = sort(unique(rrange))
    srange = sort(unique(srange))
    for h in hrange
        if (cost(h, MNV, MNV, MNV, MNV, MNV) <= MXS)
            for f in frange
                if (cost(h, f, MNV, MNV, MNV, MNV) <= MXS)
                    for l in lrange
                        if (cost(h, f, l, MNV, MNV, MNV) <= MXS)
                            for p in prange
                                if (cost(h, f, l, p, MNV, MNV) <= MXS)
                                    for r in rrange
                                        if (cost(h, f, l, p, r, MNV) <= MXS)
                                            for s in srange
                                                if (cost(h,f,l,p,r,s) <= MXS)
                                                    ff = Spud("",h,f,l,p,r,s)
                                                    score = eval_battle_list(ff, bs)
                                                    if score > bestscore
                                                        bestscore = score
                                                        bestf = ff
                                                    end
                                                end
                                            end
                                        end
                                    end                        
                                end
                            end                        
                        end
                    end
                end
            end
        end
    end
    return upgrade_spud(bestf)
end



pick_best_library (generic function with 1 method)

In [8]:
function pick_ok_counter(bs::Array{Spud})::Spud
    bestscore = -999
    bestf = Spud("",MNV,MNV,MNV,MNV,MNV,MNV)
    df = spuds_to_df(bs)
    hrange = vcat([MNV], df.f .+ 1)
    frange = vcat([MNV], df.h .+ 1)
    lrange = vcat([MNV], df.l .+ 1)
    prange = vcat([MNV], df.p .+ 1)
    rrange = vcat([MNV], df.r .+ 1)
    srange = vcat([MNV], df.s .+ 1)
    hrange = sort(unique(hrange))
    frange = sort(unique(frange))
    lrange = sort(unique(lrange))
    prange = sort(unique(prange))
    rrange = sort(unique(rrange))
    srange = sort(unique(srange))
    for h in hrange
        if (cost(h, MNV, MNV, MNV, MNV, MNV) <= MXS)
            for f in frange
                if (cost(h, f, MNV, MNV, MNV, MNV) <= MXS)
                    for l in lrange
                        if (cost(h, f, l, MNV, MNV, MNV) <= MXS)
                            for p in prange
                                if (cost(h, f, l, p, MNV, MNV) <= MXS)
                                    for r in rrange
                                        if (cost(h, f, l, p, r, MNV) <= MXS)
                                            for s in srange
                                                if (cost(h,f,l,p,r,s) <= MXS)
                                                    ff = Spud("",h,f,l,p,r,s)
                                                    score = eval_battle_list(ff, bs)
                                                    if score > bestscore
                                                        bestscore = score
                                                        bestf = ff
                                                    end
                                                end
                                            end
                                        end
                                    end                        
                                end
                            end                        
                        end
                    end
                end
            end
        end
    end
    return upgrade_spud(bestf)
end


pick_ok_counter (generic function with 1 method)

In [9]:
function pick_best_library2(bs::Array{Spud}, w::Vector{Float64})::Spud
    bestscore = -999.9
    bestf = Spud("",MNV,MNV,MNV,MNV,MNV,MNV)
    df = spuds_to_df(bs)
    hrange = vcat([MNV], df.h, df.h .+ 1, df.f, df.f .+ 1)
    frange = vcat([MNV], df.f, df.f .+ 1, df.h, df.h .+ 1)
    lrange = vcat([MNV], df.l, df.l .+ 1)
    prange = vcat([MNV], df.p, df.p .+ 1)
    rrange = vcat([MNV], df.r, df.r .+ 1)
    srange = vcat([MNV], df.s, df.s .+ 1)
    hrange = sort(unique(hrange))
    frange = sort(unique(frange))
    lrange = sort(unique(lrange))
    prange = sort(unique(prange))
    rrange = sort(unique(rrange))
    srange = sort(unique(srange))
    for h in hrange
        if (cost(h, MNV, MNV, MNV, MNV, MNV) <= MXS)
            for f in frange
                if (cost(h, f, MNV, MNV, MNV, MNV) <= MXS)
                    for l in lrange
                        if (cost(h, f, l, MNV, MNV, MNV) <= MXS)
                            for p in prange
                                if (cost(h, f, l, p, MNV, MNV) <= MXS)
                                    for r in rrange
                                        if (cost(h, f, l, p, r, MNV) <= MXS)
                                            for s in srange
                                                if (cost(h,f,l,p,r,s) <= MXS)
                                                    ff = Spud("",h,f,l,p,r,s)
                                                    score = eval_battle_list2(ff, bs, w) + 0.001 * rand()
                                                    if score > bestscore
                                                        bestscore = score
                                                        bestf = ff
                                                    end
                                                end
                                            end
                                        end
                                    end                        
                                end
                            end                        
                        end
                    end
                end
            end
        end
    end
    return upgrade_spud(bestf)
end

pick_best_library2 (generic function with 1 method)

In [10]:
function subset_library(bs::Array{Spud}, n_hits::Int, w::Vector{Float64}, thres::Float64)::Array{Spud}
    bestf = Array{Spud}(undef, n_hits)
    spud_i = 0
    df = spuds_to_df(bs)
    hrange = vcat([MNV], df.h, df.h .+ 1, df.f, df.f .+ 1)
    frange = vcat([MNV], df.f, df.f .+ 1, df.h, df.h .+ 1)
    lrange = vcat([MNV], df.l, df.l .+ 1)
    prange = vcat([MNV], df.p, df.p .+ 1)
    rrange = vcat([MNV], df.r, df.r .+ 1)
    srange = vcat([MNV], df.s, df.s .+ 1)
    hrange = sort(unique(hrange))
    frange = sort(unique(frange))
    lrange = sort(unique(lrange))
    prange = sort(unique(prange))
    rrange = sort(unique(rrange))
    srange = sort(unique(srange))
    for h in hrange
        if (cost(h, MNV, MNV, MNV, MNV, MNV) <= MXS)
            for f in frange
                if (cost(h, f, MNV, MNV, MNV, MNV) <= MXS)
                    for l in lrange
                        if (cost(h, f, l, MNV, MNV, MNV) <= MXS)
                            for p in prange
                                if (cost(h, f, l, p, MNV, MNV) <= MXS)
                                    for r in rrange
                                        if (cost(h, f, l, p, r, MNV) <= MXS)
                                            for s in srange
                                                if (cost(h,f,l,p,r,s) <= MXS)
                                                    ff = Spud("",h,f,l,p,r,s)
                                                    score = eval_battle_list2(ff, bs, w)
                                                    if score >= thres && spud_i < n_hits
                                                        spud_i += 1
                                                        bestf[spud_i] = upgrade_spud(ff)
                                                    end
                                                end
                                            end
                                        end
                                    end                        
                                end
                            end                        
                        end
                    end
                end
            end
        end
    end
    return unique(bestf[1:spud_i])
end

subset_library (generic function with 1 method)

In [11]:
# form initial library by subsampling indices
library = Array{Spud}(undef, 1000)
spud_i = 0
ss_prob = 0.01

0.01

In [12]:

hrange = MNV:MXV
frange = MNV:MXV
lrange = MNV:MXV
prange = MNV:MXV
rrange = MNV:MXV
srange = MNV:MXV

for h in hrange
    if (cost(h, MXV, MXV, MXV, MXV, MXV) >= MXS) && (cost(h, MNV, MNV, MNV, MNV, MNV) <= MXS)
        for f in frange
            if (cost(h, f, MXV, MXV, MXV, MXV) >= MXS) && (cost(h, f, MNV, MNV, MNV, MNV) <= MXS)
                for l in lrange
                    if (cost(h, f, l, MXV, MXV, MXV) >= MXS) && (cost(h, f, l, MNV, MNV, MNV) <= MXS)
                        for p in prange
                            if (cost(h, f, l, p, MXV, MXV) >= MXS) && (cost(h, f, l, p, MNV, MNV) <= MXS)
                                for r in rrange
                                    if (cost(h, f, l, p, r, MXV) >= MXS) && (cost(h, f, l, p, r, MNV) <= MXS)
                                        for s in srange
                                            if rand() < ss_prob && (cost(h,f,l,p,r,s) <= MXS)
                                                check_f = (h == MXV) || (cost(h+1,f,l,p,r,s) > MXS)
                                                check_h = (f == MXV) || (cost(h,f+1,l,p,r,s) > MXS)
                                                check_l = (l == MXV) || (cost(h,f,l+1,p,r,s) > MXS)
                                                check_prs = (p+r+s == 3*MXV) || (cost(h,f,l,p+1,r,s) > MXS)
                                                if check_h && check_f && check_l && check_prs
                                                    spud_i += 1
                                                    #randname = rand_rename(Spud(" ",h,f,l,p,r,s)).name
                                                    #name = string("#", @sprintf("%i", spud_i), ". ", randname)
                                                    name = ""
                                                    library[spud_i] = Spud(name,h,f,l,p,r,s)
                                                end
                                            end
                                        end
                                    end
                                end                        
                            end
                        end                        
                    end
                end
            end
        end
    end
end


In [13]:
library = unique(library[1:spud_i])
n_spuds = length(library)

789

In [14]:
# for i in 1:n_spuds
#     ff = library[i]
#     randname = rand_rename(ff).name
#     name = string("#", @sprintf("%i", i), ". ", randname)
#     library[i] = Spud(name,ff.h,ff.f,ff.l,ff.p,ff.r,ff.s)
# end
# id_no = Dict(library[i] => i for i in 1:n_spuds)

In [15]:
# Check that there are no ties in spudland
for iter in 1:10000
    i = rand(1:n_spuds)
    j = rand(1:n_spuds)
    if i != j && eval_battle(library[i], library[j]) == 0
        println(library[i])
        println(library[j])
        println()
    end
end

## Compute Nash env

In [16]:
function ffp(nash_env, nits)
    n_nash = length(nash_env)
    i_lose = Array{Int}(undef, (n_nash, n_nash))
    n_lose = Array{Int}(undef, n_nash)
    for i in 1:n_nash
        n_lose[i] = 0
        ff = nash_env[i]
        for j in 1:n_nash
            if eval_battle(ff, nash_env[j]) ==-1
                n_lose[i] += 1
                i_lose[i, n_lose[i]] = j
            end
        end
    end
    counts = [0 for i in 1:n_nash]
    wins = [0 for i in 1:n_nash]
    for i in 1:n_nash
        counts[i] += 1
        for j in 1:n_lose[i]
            i_w = i_lose[i, j]
            wins[i_w]+= 1
        end
    end
    for iter in 1:nits
        ind_winners = findall(wins .== maximum(wins))
        i = rand(ind_winners)
        counts[i] += 1
        for j in 1:n_lose[i]
            i_w = i_lose[i, j]
            wins[i_w]+= 1
        end
    end
    return counts
end


ffp (generic function with 1 method)

In [17]:
nash_env = [rand_rename(s) for s in library]
spuds_to_df(nash_env)

Row,name,h,f,l,p,r,s
,String,Int64,Int64,Int64,Int64,Int64,Int64
1,Detail-oriented Watchmaker Rewena bread,1,2,2,13,18,17
2,Patient Software developer Aligot,1,2,2,17,13,18
3,Inventive Etymologist Gnocchi,1,2,6,19,18,9
4,Influential Software developer Papas chorreadas,1,2,10,5,19,20
5,Motivated Actuary Rewena bread,1,2,12,7,20,16
6,Quick-thinking Paleontologist Batata vada,1,2,12,16,19,8
7,Spellbound System designer Rappie pie,1,2,12,18,9,16
8,Fair Magnate Batata vada,1,2,16,15,11,15
9,Productive Real estate investor Meat and potato pie,1,2,16,15,17,9


In [18]:
counts = ffp(nash_env, 1000);

In [19]:
nash_env = nash_env[counts .> 10]
spuds_to_df(nash_env)

Row,name,h,f,l,p,r,s
,String,Int64,Int64,Int64,Int64,Int64,Int64
1,Radiant Film producer Papas chorreadas,3,5,15,4,3,1
2,Well-versed Stewardess (or Steward) Pommes Anna,3,7,7,1,3,4
3,Distinguished Nanny (also nursemaid) Hash browns,3,8,1,1,6,2
4,Preoccupied Gladiator Meat and potato pie,4,3,15,6,1,2
5,Imaginative Shoeshiner Meat and potato pie,4,6,4,5,1,4
6,Outstanding Party-leader Cacasse à cul nu,4,6,5,4,2,3
7,Conscientious Party-leader Cacasse à cul nu,4,7,4,5,1,2
8,Absent-minded Queen dowager Rewena bread,5,1,17,3,4,3
9,Tactful Film producer Rappie pie,5,5,4,5,4,2


In [20]:
counts = ffp(nash_env, 10000)

21-element Vector{Int64}:
    1
  247
 1488
 1050
  289
    1
 1200
    1
  248
  661
    1
    1
    1
  206
  474
   21
    1
  871
 1030
  578
 1651

In [21]:
nash_env = nash_env[counts .> 10]
spuds_to_df(nash_env)

Row,name,h,f,l,p,r,s
,String,Int64,Int64,Int64,Int64,Int64,Int64
1,Well-versed Stewardess (or Steward) Pommes Anna,3,7,7,1,3,4
2,Distinguished Nanny (also nursemaid) Hash browns,3,8,1,1,6,2
3,Preoccupied Gladiator Meat and potato pie,4,3,15,6,1,2
4,Imaginative Shoeshiner Meat and potato pie,4,6,4,5,1,4
5,Conscientious Party-leader Cacasse à cul nu,4,7,4,5,1,2
6,Tactful Film producer Rappie pie,5,5,4,5,4,2
7,Assertive Party-leader Funeral potatoes,5,5,4,6,2,3
8,Selfless Novelist French fries,5,7,1,6,1,1
9,Credible Dictator Papas chorreadas,6,1,14,6,2,2


### Loop this

In [22]:
counts = ffp(nash_env, 100)

14-element Vector{Int64}:
  3
 16
 13
  3
 14
  3
  6
  4
  5
  1
 10
  9
  7
 20

In [23]:
#counts = ffp(nash_env, 10000)

In [24]:
spuds_to_df(nash_env)

Row,name,h,f,l,p,r,s
,String,Int64,Int64,Int64,Int64,Int64,Int64
1,Well-versed Stewardess (or Steward) Pommes Anna,3,7,7,1,3,4
2,Distinguished Nanny (also nursemaid) Hash browns,3,8,1,1,6,2
3,Preoccupied Gladiator Meat and potato pie,4,3,15,6,1,2
4,Imaginative Shoeshiner Meat and potato pie,4,6,4,5,1,4
5,Conscientious Party-leader Cacasse à cul nu,4,7,4,5,1,2
6,Tactful Film producer Rappie pie,5,5,4,5,4,2
7,Assertive Party-leader Funeral potatoes,5,5,4,6,2,3
8,Selfless Novelist French fries,5,7,1,6,1,1
9,Credible Dictator Papas chorreadas,6,1,14,6,2,2


In [25]:
# # don't run this every iteration!!
# nash_env = nash_env[counts .> 10]
# counts = ffp(nash_env, 10000)

In [26]:
ff = rand_rename(pick_best_library2(nash_env, counts./sum(counts)))

Spud("Understanding Teacher Rumbledethumps", 1, 8, 4, 7, 1, 3)

In [27]:
ev = eval_battle_list2(ff, nash_env, counts./sum(counts))

0.9473684210526315

In [28]:
bestf = subset_library(nash_env, 500, counts./sum(counts), ev-0.05)

1-element Vector{Spud}:
 Spud("", 1, 8, 4, 7, 1, 3)

In [29]:
cc = ffp(bestf, 10000)
println(maximum(cc))
ff2 = rand_rename(bestf[cc .== maximum(cc)][1])

10001


Spud("Skilled Shoeshiner Boxty", 1, 8, 4, 7, 1, 3)

In [30]:
eval_battle_list2(ff2, nash_env, counts./sum(counts))

0.9473684210526315

In [31]:
append!(nash_env, [ff2])
spuds_to_df(nash_env)

Row,name,h,f,l,p,r,s
,String,Int64,Int64,Int64,Int64,Int64,Int64
1,Well-versed Stewardess (or Steward) Pommes Anna,3,7,7,1,3,4
2,Distinguished Nanny (also nursemaid) Hash browns,3,8,1,1,6,2
3,Preoccupied Gladiator Meat and potato pie,4,3,15,6,1,2
4,Imaginative Shoeshiner Meat and potato pie,4,6,4,5,1,4
5,Conscientious Party-leader Cacasse à cul nu,4,7,4,5,1,2
6,Tactful Film producer Rappie pie,5,5,4,5,4,2
7,Assertive Party-leader Funeral potatoes,5,5,4,6,2,3
8,Selfless Novelist French fries,5,7,1,6,1,1
9,Credible Dictator Papas chorreadas,6,1,14,6,2,2


### loop

In [32]:
counts = ffp(nash_env, 100000)
println(maximum([eval_battle_list2(ff, nash_env, counts./sum(counts)) for ff in nash_env]))
ff = rand_rename(pick_best_library2(nash_env, counts./sum(counts)))
ev = eval_battle_list2(ff, nash_env, counts./sum(counts))
println(ev)

0.1250112483127531
0.9999200119982004


In [33]:
bestf = subset_library(nash_env, 900, counts./sum(counts), ev-0.05)

27-element Vector{Spud}:
 Spud("", 1, 8, 12, 4, 2, 4)
 Spud("", 1, 8, 4, 4, 2, 5)
 Spud("", 1, 8, 4, 4, 3, 4)
 Spud("", 1, 8, 4, 5, 2, 4)
 Spud("", 1, 9, 2, 4, 2, 4)
 Spud("", 1, 9, 1, 4, 2, 4)
 Spud("", 2, 7, 11, 2, 2, 5)
 Spud("", 2, 7, 11, 2, 3, 4)
 Spud("", 2, 7, 11, 3, 2, 4)
 Spud("", 2, 7, 15, 2, 2, 4)
 Spud("", 2, 9, 5, 2, 2, 4)
 Spud("", 2, 8, 2, 2, 2, 6)
 Spud("", 2, 9, 1, 2, 3, 4)
 ⋮
 Spud("", 2, 8, 2, 3, 3, 4)
 Spud("", 2, 8, 2, 3, 2, 5)
 Spud("", 2, 8, 2, 4, 2, 4)
 Spud("", 2, 8, 6, 2, 2, 5)
 Spud("", 2, 8, 6, 2, 3, 4)
 Spud("", 2, 8, 6, 3, 2, 4)
 Spud("", 3, 8, 4, 2, 2, 4)
 Spud("", 2, 8, 10, 2, 2, 4)
 Spud("", 3, 8, 1, 2, 2, 5)
 Spud("", 3, 8, 1, 2, 3, 4)
 Spud("", 3, 8, 1, 3, 2, 4)
 Spud("", 4, 8, 1, 2, 2, 4)

In [34]:
ff

Spud("Show-off Barman Hash browns", 3, 8, 4, 2, 2, 4)

In [35]:
ff = rand_rename(pick_best_library2(nash_env, counts./sum(counts)))
ff

Spud("Appreciative Acupuncturist Papas chorreadas", 1, 8, 4, 4, 3, 4)

In [36]:
eval_battle_list2(ff, nash_env, counts./sum(counts))

0.9999400089986503

In [37]:
ev

0.9999200119982004

In [38]:
ev

0.9999200119982004

In [39]:
println(length(bestf))
cc = ffp(bestf, 10000)
println(maximum(cc))

27
3337


In [40]:
ff2 = rand_rename(bestf[cc .== maximum(cc)][1])

Spud("Neglectful Shoeshiner Hash browns", 1, 8, 4, 5, 2, 4)

In [41]:
println(eval_battle_list2(ff2, nash_env, counts./sum(counts)))

0.9999200119982004


In [42]:
append!(nash_env, [ff2])

16-element Vector{Spud}:
 Spud("Well-versed Stewardess (or Steward) Pommes Anna", 3, 7, 7, 1, 3, 4)
 Spud("Distinguished Nanny (also nursemaid) Hash browns", 3, 8, 1, 1, 6, 2)
 Spud("Preoccupied Gladiator Meat and potato pie", 4, 3, 15, 6, 1, 2)
 Spud("Imaginative Shoeshiner Meat and potato pie", 4, 6, 4, 5, 1, 4)
 Spud("Conscientious Party-leader Cacasse à cul nu", 4, 7, 4, 5, 1, 2)
 Spud("Tactful Film producer Rappie pie", 5, 5, 4, 5, 4, 2)
 Spud("Assertive Party-leader Funeral potatoes", 5, 5, 4, 6, 2, 3)
 Spud("Selfless Novelist French fries", 5, 7, 1, 6, 1, 1)
 Spud("Credible Dictator Papas chorreadas", 6, 1, 14, 6, 2, 2)
 Spud("Attentive Juggler Clapshot", 6, 5, 6, 3, 1, 2)
 Spud("Careless Investment broker Gamjajeon", 7, 1, 11, 3, 11, 2)
 Spud("Brilliant Novelist Gnocchi", 7, 5, 2, 6, 3, 1)
 Spud("Hospitable Laundress (also Lavendar) Stoemp", 7, 6, 1, 2, 1, 5)
 Spud("Unenthused Film producer Suan la tu dou si", 8, 1, 10, 3, 6, 3)
 Spud("Skilled Shoeshiner Boxty", 1, 8, 4, 7, 1, 

In [43]:
for ii in 1:200
    #println()
    print(ii)
    counts = ffp(nash_env, 100000)
    #println(maximum([eval_battle_list2(ff, nash_env, counts./sum(counts)) for ff in nash_env]))
    ff = rand_rename(pick_best_library2(nash_env, counts./sum(counts)))
    ev = eval_battle_list2(ff, nash_env, counts./sum(counts))
    #println(ev)
    bestf = subset_library(nash_env, 900, counts./sum(counts), ev-0.05)
    #println(length(bestf))
    cc = ffp(bestf, 10000)
    #println(maximum(cc))
    ff2 = rand_rename(bestf[cc .== maximum(cc)][1])
    #println(eval_battle_list2(ff2, nash_env, counts./sum(counts)))
    append!(nash_env, [ff2])
end
println()
counts = ffp(nash_env, 100000)
#ff = pick_best_library2(nash_env, counts./sum(counts))
#eval_battle_list2(ff, nash_env, counts./sum(counts))    
nash_env = nash_env[counts .> 10]
counts = ffp(nash_env, 100000);

ff = pick_best_library2(nash_env, counts./sum(counts))
eval_battle_list2(ff, nash_env, counts./sum(counts))

123456789101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828384858687888990919293949596979899100101102103104105106107108109110111112113114115116117118119120121122123124125126127128129130131132133134135136137138139140141142143144145146147148149150151152153154155156157158159160161162163164165166167168169170171172173174175176177178179180181182183184185186187188189190191192193194195196197198199200


0.0644043422248409

In [44]:
nash_env

133-element Vector{Spud}:
 Spud("Distinguished Nanny (also nursemaid) Hash browns", 3, 8, 1, 1, 6, 2)
 Spud("Unflinching Ferryman Funeral potatoes", 9, 6, 3, 1, 1, 1)
 Spud("Methodical Juggler Suan la tu dou si", 7, 2, 9, 7, 2, 2)
 Spud("Crafty Automobile racer Potatonik", 7, 1, 11, 9, 2, 5)
 Spud("Fierce Consultant Pommes Anna", 2, 5, 12, 3, 6, 4)
 Spud("Covert Housekeeper Rappie pie", 7, 7, 3, 2, 1, 1)
 Spud("Composed Paramedic Tartiflette", 3, 10, 3, 3, 2, 1)
 Spud("Reckless Environmental scientist Funeral potatoes", 2, 7, 1, 7, 4, 1)
 Spud("Unremarkable Dictator Batata vada", 11, 1, 7, 8, 3, 1)
 Spud("Lackadaisical Shoeshiner Rappie pie", 5, 10, 2, 2, 1, 1)
 Spud("Determined Gondolier Älplermagronen", 2, 5, 15, 6, 1, 5)
 Spud("Unimpressed Tutor Pommes Anna", 5, 2, 12, 1, 7, 7)
 Spud("Uninspired Magnate Clapshot", 7, 3, 7, 1, 5, 4)
 ⋮
 Spud("Tender Auctioneer Okroshka", 2, 5, 15, 8, 2, 2)
 Spud("Incomprehensible Chimney sweep Ajiaco", 12, 3, 4, 1, 2, 2)
 Spud("Withdrawn Stewardess (

## beware

In [45]:
CSV.write("spuds_nash_stage01.csv", spuds_to_df(nash_env))

"spuds_nash_stage01.csv"

### refine weights

In [ ]:
eps = 0.01/length(library)
nits = 40000
sub_iter = 2000
w = counts./sum(counts)

In [ ]:
n_nash = length(nash_env)
i_lose = Array{Int}(undef, (n_nash, n_nash))
n_lose = Array{Int}(undef, n_nash)
for i in 1:n_nash
    n_lose[i] = 0
    ff = nash_env[i]
    for j in 1:n_nash
        if eval_battle(ff, nash_env[j]) ==-1
            n_lose[i] += 1
            i_lose[i, n_lose[i]] = j
        end
    end
end

In [ ]:
scores = [0.0 for i in 1:n_nash]
for i in 1:n_nash
    for j in 1:n_lose[i]
        i_w = i_lose[i, j]
        scores[i_w]+= w[i]
    end
end

In [ ]:
previous_score = maximum(scores)
for iter in 1:nits
    ind_winners = findall(scores .== maximum(scores))
    i = rand(ind_winners)
    w[i] += eps
    for j in 1:n_lose[i]
        i_w = i_lose[i, j]
        scores[i_w]+= eps
    end
    if mod1(iter, sub_iter) == 1
        scores = scores./sum(w)
        w = w./sum(w)
        if maximum(scores) > previous_score
            eps = eps/2
            println(eps)
            println()
            println()
        end
        previous_score = maximum(scores)
        mxv, mxi = findmax(scores)
        print(nash_env[mxi])
        print(" w:")
        print(w[mxi])
        print(" s:")
        print(scores[mxi])
        println()
    end
end


In [ ]:
maximum([eval_battle_list2(ff, nash_env, w) for ff in nash_env])

In [ ]:
ff = pick_best_library2(nash_env, w)
println(ff)
eval_battle_list2(ff, nash_env, w)

## Old code

In [ ]:
function nashprox2(size, start_size, ns)
    nash_env = Array{Spud}(undef, size)
    nash_env[1:start_size] = random_team(() -> rand(library), start_size)
    for j in (start_size+1):size
        if j < ns
            ff = pick_ok_counter(nash_env[1:(j-1)])
            nash_env[j] = ff
        else
            ff = pick_ok_counter(randsubseq(nash_env[1:(j-1)], ns/j))
            nash_env[j] = ff                
        end
    end
    return nash_env
end

function nashprox(size, start_size, ns)
    nash_env = Array{Spud}(undef, size)
    nash_env[1:start_size] = random_team(() -> rand(library), start_size)
    for j in (start_size+1):size
        if j < ns
            ff = pick_best_library(nash_env[1:(j-1)])
            nash_env[j] = ff
        else
            ff = pick_best_library(randsubseq(nash_env[1:(j-1)], ns/j))
            nash_env[j] = ff                
        end
    end
    return nash_env
end

In [ ]:
@time nash_env = nashprox(4000, 10, 100)
nash_env = unique(nash_env)
#inds_nash = [id_no[ff] for ff in nash_env];
#length(inds_nash)
n_nash = length(nash_env)

In [ ]:
# Verify 3-explotability
for i in 1:1000
    #inds = sort([rand(inds_nash) for i in 1:3])
    #rt = library[inds]
    rt = random_team(() -> rand(nash_env), 3)
    ff = pick_best_library(rt)
    ev = eval_battle_list(ff, rt)
    if ev < 3
        println(rt)
    end
end


In [ ]:
# Verify 4-explotability
for i in 1:1000
    #inds = sort([rand(inds_nash) for i in 1:3])
    #rt = library[inds]
    rt = random_team(() -> rand(nash_env), 4)
    ff = pick_best_library(rt)
    ev = eval_battle_list(ff, rt)
    if ev < 4
        println(rt)
        println(ev)
    end
end


In [ ]:
#inds = sort([rand(1:n_spuds) for i in 1:5])
#rt = library[inds]

In [ ]:
# rt = library[[1403,1902,2283,2397]]
# rt

In [ ]:
# ff = pick_best(library, rt)
# ind = [i for i in 1:n_spuds if library[i] == ff][1]
# print(ff)
# print(" ")
# eval_battle_list(ff, rt)

## Fictitious Play to compute Nash distribution

In [ ]:
@time nash_env = nashprox(4000, 10, 100)
nash_env = unique(nash_env)
#inds_nash = [id_no[ff] for ff in nash_env];
#length(inds_nash)
n_nash = length(nash_env)

In [ ]:
i_lose = Array{Int}(undef, (n_nash, n_nash))
n_lose = Array{Int}(undef, n_nash)
for i in 1:n_nash
    n_lose[i] = 0
    ff = nash_env[i]
    for j in 1:n_nash
        if eval_battle(ff, nash_env[j]) ==-1
            n_lose[i] += 1
            i_lose[i, n_lose[i]] = j
        end
    end
end

In [ ]:
# initialize FP with 1 of each
counts = [0 for i in 1:n_nash]
wins = [0 for i in 1:n_nash]
for i in 1:n_nash
    counts[i] += 1
    for j in 1:n_lose[i]
        i_w = i_lose[i, j]
        wins[i_w]+= 1
    end
end

In [ ]:
nits = 500000
for iter in 1:nits
    ind_winners = findall(wins .== maximum(wins))
    for i in ind_winners
        counts[i] += 1
        for j in 1:n_lose[i]
            i_w = i_lose[i, j]
            wins[i_w]+= 1
        end
    end
    if mod1(iter, 10000) == 1
        mxv, mxi = findmax(wins)
        print(nash_env[mxi])
        print(" n:")
        print(counts[mxi])
        print(" w:")
        print(wins[mxi])
        println()
    end
end

w = counts./sum(counts)
println(maximum([eval_battle_list2(ff, nash_env, w) for ff in nash_env]))
sum(counts)

In [ ]:
ff = pick_best_library2(nash_env, w)
println(ff)
eval_battle_list2(ff, nash_env, w)

In [ ]:
inds_sel = sortperm(-w)[1:500]
tab = spuds_to_df(nash_env[inds_sel])
tab[!, :w] = w[inds_sel]
tab

In [ ]:
# Verify explotability
for i in 1:10000
    inds = sort([rand(inds_sel[1:120]) for i in 1:3])
    rt = nash_env[inds]
    ff = pick_best(library, rt)
    ev = eval_battle_list(ff, rt)
    if ev < 3
        println(spuds_to_df(rt))
        println(w[inds])
        println(ev)
    end
end

In [ ]:
rt = nash_env[sort([rand(1:n_nash) for i in 1:4])]

In [ ]:
ff = pick_best_rdmly(nash_env[inds_sel], rt, 99)
ev = eval_battle_list(ff, rt)
println(ff)
println(ev)
eval_battle_list2(ff, nash_env, w)

In [ ]:
ff = pick_best_rdmly(library, rt, 99)
ev = eval_battle_list(ff, rt)
println(ff)
println(ev)
eval_battle_list2(ff, nash_env, w)

In [ ]:
ff = rand(library)
println(ff)
eval_battle_list2(ff, nash_env, w)

In [ ]:
#cost(4,4,9,1,4,2)

In [ ]:
#rt = library[[15918,21020,26191]]
#eval_battle(Spud("",4,4,9,1,4,1), rt[1])

In [ ]:
#sum(sort(-w)[1:450])

In [ ]:
#sum(w)

In [ ]:
# # FP is slow to converge in the end
# eps = 0.01/length(library)
# nits = 20000
# scores = wins./sum(counts)
# w = counts./sum(counts)
# 0

In [ ]:
# previous_score = maximum(scores)
# for iter in 1:nits
#     ind_winners = findall(wins .== maximum(wins))
#     i = rand(ind_winners)
#     w[i] += eps
#     for j in 1:n_lose[i]
#         i_w = i_lose[i, j]
#         scores[i_w]+= eps
#     end
#     if mod1(iter, 2000) == 1
#         scores = scores./sum(w)
#         w = w./sum(w)
#         if maximum(scores) > previous_score
#             eps = eps/2
#             println(eps)
#             println()
#             println()
#         end
#         previous_score = maximum(scores)
#         mxv, mxi = findmax(wins)
#         print(library[mxi])
#         print(" n:")
#         print(w[mxi])
#         print(" w:")
#         print(scores[mxi])
#         println()
#         mxv, mxi = findmax(counts)
#         print(library[mxi])
#         print(" n:")
#         print(w[mxi])
#         print(" w:")
#         print(scores[mxi])
#         println()
#         println(maximum([eval_battle_list2(ff, library, w) for ff in library]))
#         println()
#     end
# end
